MÁSTER BIG DATA & DATA ENGINEERING

Entrega Final Módulo: Programación avanzada en Python

Autor: Germán Artigot Cortizo



## Nota Preliminar

Según se podrá observar en la evaluación de este proyecto, a lo largo del mismo se recurren a distintos métodos para realizar funciones similares. Esto no es sino debido a que el proyecto se ha realizado a lo largo de varias semanas por lo que, según se profundizaba en el conocimiento y la práctica de la programación con Pandas, se han hallado diversas formas de lograr el mismo resultado.


## Introducción

En el portal de datos abiertos de la EMT de Madrid se puede encontrar abundante información sobre transporte y movilidad de la ciudad. En particular los datos relativos a la utilización del servicio de bicicletas eléctricas del ayuntamiento de Madrid.
La dirección del portal mencionado es la siguiente:

https://opendata.emtmadrid.es/Home

y el enlace donde se encuentran los datos relativos a BiciMAD es el siguiente:

https://opendata.emtmadrid.es/Datos-estaticos/Datos-generales-(1)

En esta página podemos encontrar los datos de uso de BiciMAD por meses desde abril de 2019 hasta el mes Febrero de 2023. Se trata de un conjunto de URLs a ficheros comprimidos ZIP, donde una vez descomprimidos, podemos encontrar un fichero en formato `csv`.
En esta práctica __solo nos va a interesar los datos de uso desde junio el 2021__. Son ficheros con nombre:

`
trips_YY_MM_monthName.csv
`

donde `YY` representa el año, `MM` representa el mes y `monthName` representa el nombre del mes en inglés.

Los datos del mes 10 del año 2021 está mal generado por la EMT, por lo que se pide que no se use ese fichero.

### Metadatos

En cada uno de los ficheros `csv` podemos encontrar los siguientes datos:

* `date` (Fecha): La fecha en la que tuvo lugar el viaje.
* `idbike` (ID de Bicicleta): El identificador único de la bicicleta utilizada para el viaje.
* `fleet` (Flota): La flota a la que pertenece la bicicleta utilizada.
* `trip_minutes` (Duración del Viaje en Minutos): La duración del viaje en minutos.
* `geolocation_unlock` (Geolocalización de Desbloqueo): Las coordenadas geográficas del lugar de inicio del viaje.
* `address_unlock` (Dirección de Desbloqueo): La dirección postal en la que se desbloqueó la bicicleta.
* `unlock_date` (Fecha y Hora de Desbloqueo): La fecha y hora exactas en las que comenzó el viaje.
* `locktype` (Tipo de Bloqueo): El estado de la bicicleta antes del viaje. Puede estar anclada en una estación o simplemente bloqueada en cualquier lugar libremente.
* `unlocktype` (Tipo de Desbloqueo): El estado de la bicicleta después del viaje.
* `geolocation_lock` (Geolocalización de Bloqueo): Las coordenadas geográficas del lugar final del viaje.
* `address_lock` (Dirección de Bloqueo): La dirección postal en la que se bloqueó la bicicleta.
* `lock_date` (Fecha y Hora de Bloqueo): La fecha y hora exactas en las que finalizó el viaje.
* `station_unlock` (Número de Estación de Desbloqueo): El número de la estación en la que la bicicleta estaba anclada antes del viaje, si la había.
* `dock_unlock` (Muelle de Desbloqueo): El muelle de la estación en la que la bicicleta estaba anclada antes del viaje, si la había.
*  `unlock_station_name` (Nombre de la Estación de Desbloqueo): El nombre de la estación en la que la bicicleta estaba anclada antes del viaje, si la había.
* `station_lock` (Número de Estación de Bloqueo): El número de la estación en la que la bicicleta estaba anclada después del viaje, si la había.
* `dock_lock` (Muelle de Bloqueo): El muelle de la estación en la que la bicicleta estaba anclada después del viaje, si la había.
* `lock_station_name` (Nombre de la Estación de Bloqueo): El nombre de la estación en la que la bicicleta estaba anclada después del viaje, si la había.

## Librerías

Importar todas las librerías necesarias al principio, tal como propone el estilo `pep-8`. Ej.:
<a href="https://www.python.org/dev/peps/pep-0008/">PEP 8 -- Style Guide for Python Code</a>.

In [1]:
#Importación de librerías necesarias
from BiciMad import urlemt
import io
import re
import requests
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt

## ETAPA 1. Análisis exploratorio y consultas.




### La descarga de los ficheros

En este notebook vamos a estudiar únicamente los datos de febrero de 2023.
La URL que permite descargar dichos datos es la siguiente:

In [2]:
url = "https://opendata.emtmadrid.es/getattachment/7a88cb04-9007-4520-88c5-a94c71a0b925/trips_23_02_February-csv.aspx"

Observa que en la URL aparece la cadena `trips_23_02`, y que el fichero ZIP tiene un archivo de datos en formato csv llamado `trips_23_02_February.csv`.


__Se pide:__
 Escribir una función llamada `csv_from_zip` que reciba como único argumento un string que represente la URL al fichero ZIP de datos de uso y devuelva un fichero en formato CSV. El tipo del objeto devuelto es `TextIO`. La función  lanzará una excepción de tipo `ConnectionError` en caso de que falle la petición al servidor de la EMT.

__Observaciones:__
En el notebook `05_ficheros` del módulo de `Secuencias` se habla del manejo de ficheros en formato ZIP, y de cómo leer un fichero, transformarlo en una cadena de caracteres para posteriormente, usando el método `String.IO`, obtener un fichero de texto.
Deberás importar el módulo `typing`.

In [3]:
# Solución 1: Haciendo uso del método con la misma funcionalidad que definimos en la clase UrlEmt
def csv_from_zip(url: str) -> io.StringIO:
    """
    Función que devuelva los datos de la EMT para un URL suministrado
    :param url: URL de acceso a los datos de la EMT
    :return: contenido del CSV producto de la descarga
    """
    ex = r'_\d{2}_\d{2}_'
    pattern = re.compile(ex)
    year_month = re.search(pattern, url)[0]
    year_month_list = year_month.split('_')
    year_month_list = list(filter(None, year_month_list))

    emt_info = urlemt.UrlEmt()
    return emt_info.get_csv(int(year_month_list[1]), int(year_month_list[0]))

In [6]:
# Prueba
csvfile = csv_from_zip(url)
print(f'El tipo de la variable devuelta por la función es: {type(csvfile)}')

El tipo de la variable devuelta por la función es: <class '_io.TextIOWrapper'>


In [10]:
#Solución 2: Desgranando la lógica del método invocado en la solución 1
def csv_from_zip(url: str) -> io.StringIO:
    """
    Función que devuelva los datos de la EMT para un URL suministrado
    :param url: URL de acceso a los datos de la EMT
    :return: contenido del CSV producto de la descarga
    """
    ex = r'_\d{2}_\d{2}_'
    pattern = re.compile(ex)
    year_month = re.search(pattern, url)[0]
    year_month_list = year_month.split('_')
    year_month_list = list(filter(None, year_month_list))
    
    try:
        url_connection = requests.get(url)
        url_connection.raise_for_status()
    except ValueError as error:
        raise error
    except Exception:
        raise ConnectionError(f'No se pudo conectar correctamente al servidor de la EMT.'
                              f'Code: f{url_connection.status_code}. Error: f{url_connection.reason}')
    
    filename = f'trips_{year_month_list[0]}_{year_month_list[1]}'
    zip_file = zipfile.ZipFile(io.BytesIO(url_connection.content))
    zip_file.extractall(r'./data/' + filename)
    zip_content = os.listdir(r'./data/' + filename)
    csv_file = list(filter(lambda f: f.endswith('.csv'), zip_content))[0]
    csv = open(r'./data/' + filename  + r'/' + csv_file, 'r', encoding='utf-8')
    return csv

In [11]:
# Prueba
csvfile = csv_from_zip(url)
print(f'El tipo de la variable devuelta por la función es: {type(csvfile)}')

El tipo de la variable devuelta por la función es: <class '_io.TextIOWrapper'>


### Creación del dataframe de datos

__Se pide:__
Crear una función llamada `get_data` que reciba como único argumento un fichero csv y que devuelva un objeto de tipo DataFrame con las siguientes restricciones:
* El índice del dataframe debe ser a fecha en la que tuvo lugar el viaje (usa el argumento `index_col`).
* Todas las fechas que aparecen en los datos ha de tener tipo `datetime` (usa el argumento 'parse_dates`).
* De todas las columnas que hay en el fichero, solo nos interesan las siguientes:

   ```
     [ 'idBike', 'fleet', 'trip_minutes', 'geolocation_unlock', 'address_unlock', 'unlock_date', 'locktype',
     'unlocktype', 'geolocation_lock', 'address_lock', 'lock_date', 'station_unlock', 'unlock_station_name',
     'station_lock', 'lock_station_name']
   ```

por lo que el dataframe ha de tener solo dichas columnas.

__Observaciones:__ Observa que los datos de usos que publica la EMT son fichero CSV donde el separador de campos es el símbolo `;`.

In [12]:
# Solución:
def get_data(csv: io.StringIO) -> pd.DataFrame:
    """
    Función que dados los datos contenidos en un CSV devuelve un objeto pandas.DataFrame con los datos formateados debidamente
    :param csv: stream de lectura de los datos contenidos en el CSV
    :return: objeto pandas.DatFrame con los datos con fechas en el tipo de dato debido e índice fecha
    """
    data_df = pd.read_csv(csv, sep=';', index_col='fecha', parse_dates=['fecha', 'unlock_date', 'lock_date'])

    return data_df

In [28]:
# Prueba
csvfile = csv_from_zip(url)
usos = get_data(csvfile)
usos.head()
print(f'El índice del DataFrame es: {usos.index.name}')
print(f'Los tipos de dato de las columnas de fechas son: {usos.index.dtype},{usos.unlock_date.dtype}, {usos.lock_date.dtype}')
print(f'Y las columnas son: {usos.columns.values}')

El índice del DataFrame es: fecha
Los tipos de dato de las columnas de fechas son: datetime64[ns],datetime64[ns], datetime64[ns]
Y las columnas son: ['idBike' 'fleet' 'trip_minutes' 'geolocation_unlock' 'address_unlock'
 'unlock_date' 'locktype' 'unlocktype' 'geolocation_lock' 'address_lock'
 'lock_date' 'station_unlock' 'dock_unlock' 'unlock_station_name'
 'station_lock' 'dock_lock' 'lock_station_name']


### Explorar la estructura de los datos en el dataframe

El método `info` devuelve información acerca de la estructura de un dataframe.

__Se pide:__
Escribe el código necesario para crear un dataframe llamado `usos` a partir de la url "https://opendata.emtmadrid.es/getattachment/7a88cb04-9007-4520-88c5-a94c71a0b925/trips_23_02_February-csv.aspx".
Ejecuta el método `info()` sobre el dataframe y comenta los resultados.
Fíjate en las columnas que representan fechas y comprueba su tipo de datos. ¿Cuántas filas se han cargado?. Fíjate en los valores distintos de `NaN` en cada una de las columnas.

In [47]:
# Solución:
url_csv = csv_from_zip(url)
usos = get_data(url_csv)
usos.info()

print('\n' + f'El número de columnas cargadas en el DataFrame (incluyendo índice) es: {usos.shape[1] + usos.index.nlevels}')
print(f'El tipo de dato de las columnas de fecha es: "fecha"->{usos.index.dtype} "unlock_date"->{usos.unlock_date.dtype} "lock_date"->{usos.lock_date.dtype}')
print(f'El número de filas cargadas en el DataFrame (incluyendo índice) es: {len(usos.index)}')
print('El número de valores NaN en las columnas es:\n' + f'{usos.isna().sum()}')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336988 entries, NaT to 2023-02-18
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   idBike               168494 non-null  float64       
 1   fleet                168494 non-null  float64       
 2   trip_minutes         168494 non-null  float64       
 3   geolocation_unlock   168494 non-null  object        
 4   address_unlock       168494 non-null  object        
 5   unlock_date          168494 non-null  datetime64[ns]
 6   locktype             168494 non-null  object        
 7   unlocktype           168494 non-null  object        
 8   geolocation_lock     168494 non-null  object        
 9   address_lock         168494 non-null  object        
 10  lock_date            168494 non-null  datetime64[ns]
 11  station_unlock       168114 non-null  float64       
 12  dock_unlock          168114 non-null  float64       
 13  unloc

### Borrado de valores NaN.

El método  `dropna` de los DataFrames tiene un argumento `how` que permite indicar que se borren filas o columnas donde todos los valores son `NaN`.

__Se pide:__ Crear una función llamada `delete_nan_rows` que borre las filas con todos sus valores `NaN` de un dataframe recibido como argumento. La función modifica el dataframe y no devuelve nada.  

Comenta los resultados. Prueba la función `delete_nan_rows` con el dataframe `usos`. Comprueba que el número de filas del dataframe `usos` ha cambiado. Puedes volver a ejecutar el método `info` y compáralo con el resultado antes de invocar a la función `delete_nan_rows`.

In [49]:
# Solución:
def delete_nan_rows(df: pd.DataFrame) -> None:
    """
    Función para eliminar todos los registros de un pandas.DataFrame que tengan todos sus valores NaN
    :param df: objeto pandas.DataFrame de entrada
    :return: no hay salida, el objeto pandas.DataFrame de entrada es directamente modificado eliminando registros NaN
    """
    df.dropna(axis=0, how='all', inplace=True)

In [51]:
# Prueba
url_csv = csv_from_zip(url)
usos = get_data(url_csv)
delete_nan_rows(usos)
usos.info()

print('\nEl número de valores NaN en las columnas es:\n' + f'{usos.isna().sum()}')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 168494 entries, 2023-02-01 to 2023-02-18
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   idBike               168494 non-null  float64       
 1   fleet                168494 non-null  float64       
 2   trip_minutes         168494 non-null  float64       
 3   geolocation_unlock   168494 non-null  object        
 4   address_unlock       168494 non-null  object        
 5   unlock_date          168494 non-null  datetime64[ns]
 6   locktype             168494 non-null  object        
 7   unlocktype           168494 non-null  object        
 8   geolocation_lock     168494 non-null  object        
 9   address_lock         168494 non-null  object        
 10  lock_date            168494 non-null  datetime64[ns]
 11  station_unlock       168114 non-null  float64       
 12  dock_unlock          168114 non-null  float64       
 13

###  Exploración de las columnas.

La columna `fleet` representa el tipo de flota de bicicleta eléctrica.
Las BiciMAD Go son free floating y se corresponden con tipo de flota 2. A diferencia de las BiciMAD normales, no tienen por qué dejarse en una estación de bicicletas, aunque existe la posibilidad de hacerlo. Su bloqueo se realiza por medio de un candado manual, aspecto en el que se diferencian de las otras bicis municipales.

Por otro lado, las columnas `locktype` y `unlocktype` tienen información del tipo de bloqueo y desbloqueo respectivamente.

__Se pide:__
* ¿Cuántos valores distintos hay en la columna `fleet` del dataframe `usos`?
* ¿Cuántos valores distintos hay en las columnas `locktype` y `unlocktype` del dataframe `usos`?

In [54]:
# Solución:
url_csv = csv_from_zip(url)
usos = get_data(url_csv)
delete_nan_rows(usos)

valores_fleet = usos['fleet'].unique()           # Valores distintos de la columna fleet
valores_locktype = usos['locktype'].unique()     # Valores distintos de la columna locktype
valores_unlocktype = usos['unlocktype'].unique() # Valores distintos de la columna unlocktype

print(f'El número de valores distintos de la columna fleet es: {len(valores_fleet)}')
print(f'El número de valores distintos de la columna locktype es: {len(valores_locktype)}')
print(f'El número de valores distintos de la columna unlocktype es: {len(valores_unlocktype)}')

El número de valores distintos de la columna fleet es: 2
El número de valores distintos de la columna locktype es: 2
El número de valores distintos de la columna unlocktype es: 2


### Cambio de tipo de datos

Las columnas  `idBike` y `fleet`  tienen tipo float, pero en realidad representan un identificador de bicicleta y de flota respectivamente.

__Se pide:__ Crear una función llamada `float_to_str` que cambie el tipo de una columna a tipo string. La función recibe como argumentos un dataframe y el nombre de una columna. La función no devuelve nada.
Si la columna no se encuentra entre las columnas del dataframe, la función no hace nada.

__Observaciones:__ Puedes usar la función `map` para aplicar la función de transformación.

Prueba la función con el datafame `usos` y  las columnas `idBike` y `fleet`.
De esta forma, el identificador de bicicleta `7337.0` pasará a ser `'7337'`. Ejecuta la función `info` sobre el dataframe `usos` para comprobar que los tipos asociados a esas columnas han cambiado.

In [56]:
# Solución:
def float_to_str(df: pd.DataFrame, col:str) -> None:
    """
    Función para que castea la columna suministrada a tipo string
    :param df: objeto pandas.DataFrame contenedor de la columna a modificar
    :param col: columna que se desea castear
    :return: no hay salida, el parámetro df suministrado a la entrada es modificado adhoc
    """
    df[col] = df[col].map(lambda x: str(x))

In [57]:
# Prueba
url_csv = csv_from_zip(url)
usos = get_data(url_csv)
print(f'El tipo de datos de las columnas idBike y fleet antes del cambio son: {usos.idBike.dtype} y {usos.fleet.dtype} respectivamente')

float_to_str(usos, 'idBike')
float_to_str(usos, 'fleet')
print(f'El tipo de datos de las columnas idBike y fleet despues del cambio son: {usos.idBike.dtype} y {usos.fleet.dtype} respectivamente')
print('\nRecordemos que en pandas ya que los string pueden tener longitudes distintas el tipo de dato por defecto para strings es "object" para ajustarse a todas las longitudes.')

El tipo de datos de las columnas idBike y fleet antes del cambio son: float64 y float64 respectivamente
El tipo de datos de las columnas idBike y fleet despues del cambio son: object y object respectivamente

Recordemos que en pandas ya que los string pueden tener longitudes distintas el tipo de dato por defecto para strings es "object" para ajustarse a todas las longitudes.


## Consultas

Las siguientes consultas se realizarán con el dataframe `usos` creado en los apartados anteriores.

### C1

¿Cuántas bicicletas han sido desbloqueadas de una estación y no se bloquean en ninguna?

In [ ]:
# Solución:
filtro_c1 = (usos['locktype'] == 'STATION') & (usos['unlocktype'] == 'FREE')
usos_c1 = usos.where(filtro_c1)

In [ ]:
# Prueba


### C2

Seleccionar solo las bicicletas del tipo de flota '1' . El nuevo dataframe se ha de llamar `regular_fleet`.

In [ ]:
# Solución:

In [ ]:
# Prueba:

### C3

Crea una función llamada `day_time` para calcular las horas totales de uso de bicicletas por día del mes.
La función recibe un dataframe y devuelve una Serie donde el índice es la fecha y el valor es el número de horas.

Prueba la función anterior con el dataframe  `usos`.

__Opcional:__ Dibuja un diagrama de barras con el resultado de la función `day_time` sobre el dataframe `usos`. Utiliza el módulo `matplotlib.pyplot`.

In [ ]:
# Solución:
def day_time(df: pd.DataFrame) -> pd.Series:
    """
    Función para calcular el uso en horas de bicicletas por día del mes
    :param df: objeto pandas.DataFrame con los datos de uso de la EMT por mes
    :return: objeto pandas.Series con el día y número de horas de uso de las bicicletas
    """
    data_dates = df.index.unique()
    trips_per_day = pd.Series()
    for date in data_dates:
        trips_day = pd.Series(df[df.index == date]['trip_minutes'].sum()/60, index=[date])
        trips_per_day= pd.concat([trips_per_day, trips_day])
    return trips_per_day

In [ ]:
# Prueba:
use_hours_date = day_time(usos)
use_hours_date.plot.bar()
plt.show()

### C4

Crea una función llamada `weekday_time` para calcular las horas totales de uso de bicicletas por día de la semana. La función recibe un dataframe y devuelve una Serie donde el índice es el día de la semana (L, M, X, J, V, S, D) y el valor es el número de horas.

Prueba la función anterior con el dataframe usos.



In [ ]:
# Solución:


In [ ]:
# Prueba


### C5

Crea una función llamada `total_usage_day` para calcular el número total de usos de bicicletas por día del mes. La función recibe un dataframe y devuelve una Serie donde el índice es la fecha y el valor es el número de usos.

Prueba la función anterior con el dataframe usos.



In [ ]:
# Solución:


In [ ]:
# Prueba


### C6

Calcular el total de usos por fecha y estación de desbloqueo. Como la fecha está en el índice y la estación de desbloqueo es una columna, lo mejor es agrupar por la columna `station_unlock` y  un objeto `Grouper` de frecuencia diaria (1D).

A continuación, se muestra un ejemplo del funcionamiento de la función Grouper de pandas:
```
>>> df
             ID  Price
2000-01-02   0     10
2000-01-02   1     20
2000-01-09   2     30
2000-01-16   3     40

# Grouper con frecuencia semanal (1W)
>>> df.groupby(pd.Grouper(freq="1W")).mean()
               ID  Price

2000-01-02    0.5   15.0
2000-01-09    2.0   30.0
2000-01-16    3.0   40.0
```

In [ ]:
# Solución:


In [ ]:
# Prueba


### C7

Se quiere averiguar la dirección de las estaciones de desbloqueo que a lo largo del mes han tenido un mayor número de viajes. Crear una función llamada `most_popular_stations` que calcule dicho dato. El argumento de la función es un dataframe y devuelve un conjunto con la dirección de las estaciones.



In [ ]:
# Solución:


In [ ]:
# Prueba


### C8

Se quiere averiguar el  número de usos de las estaciones de desbloqueo que a lo largo del mes ha tenido un mayor número de viajes. Crear una función llamada `usage_from_most_popular_station` que calcule dicho dato. El argumento de la función es un dataframe y devuelve un entero.

In [ ]:
# Solución:


In [ ]:
# Prueba
